# Notebook for testing out code segments


In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [7]:
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import tensorflow_models as tfm
import matplotlib.pyplot as plt

from PIL import Image

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

/home/darshk/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [8]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if 'GPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'GPU'
elif 'TPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'TPU'
else:
  print('Running on CPU is slow, so only train for a few steps.')
  device = 'CPU'

device

This may be broken in Colab.


'GPU'

In [9]:
expConfig = exp_factory.get_exp_config('retinanet_resnetfpn_coco')

In [10]:
batch_size = 8
num_classes = 4

HEIGHT, WIDTH = 256, 256
IMG_SIZE = [HEIGHT, WIDTH, 3]

# Backbone config.
expConfig.task.freeze_backbone = False
expConfig.task.annotation_file = ''

# Model config.
expConfig.task.model.input_size = IMG_SIZE
expConfig.task.model.num_classes = num_classes
expConfig.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = expConfig.task.model.num_classes

# Training data config.
expConfig.task.train_data.input_path = "../Dataset/road_coco/records/train-00000-of-00001.tfrecord"
expConfig.task.train_data.dtype = 'float32'
expConfig.task.train_data.global_batch_size = batch_size
expConfig.task.train_data.parser.aug_scale_max = 1.0
expConfig.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
expConfig.task.validation_data.input_path = "../Dataset/road_coco/records/valid-00000-of-00001.tfrecord"
expConfig.task.validation_data.dtype = 'float32'
expConfig.task.validation_data.global_batch_size = batch_size

In [11]:
train_steps = 1000
expConfig.trainer.steps_per_loop = 100 # steps_per_loop = num_of_training_examples // train_batch_size

expConfig.trainer.summary_interval = 100
expConfig.trainer.checkpoint_interval = 100
expConfig.trainer.validation_interval = 100
expConfig.trainer.validation_steps =  100 # validation_steps = num_of_validation_examples // eval_batch_size
expConfig.trainer.train_steps = train_steps
expConfig.trainer.optimizer_config.warmup.linear.warmup_steps = 100
expConfig.trainer.optimizer_config.learning_rate.type = 'cosine'
expConfig.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
expConfig.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
expConfig.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

In [12]:
pp.pprint(expConfig.as_dict())

{   'runtime': {   'all_reduce_alg': None,
                   'batchnorm_spatial_persistent': False,
                   'dataset_num_private_threads': None,
                   'default_shard_dim': -1,
                   'distribution_strategy': 'mirrored',
                   'enable_xla': False,
                   'gpu_thread_mode': None,
                   'loss_scale': None,
                   'mixed_precision_dtype': 'bfloat16',
                   'num_cores_per_replica': 1,
                   'num_gpus': 0,
                   'num_packs': 1,
                   'per_gpu_thread_count': 0,
                   'run_eagerly': False,
                   'task_index': -1,
                   'tpu': None,
                   'tpu_enable_xla_dynamic_padder': None,
                   'worker_hosts': None},
    'task': {   'allow_image_summary': False,
                'annotation_file': '',
                'differential_privacy_config': None,
                'export_config': {   'cast_detection_c

In [13]:
if expConfig.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    
distStrat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [14]:
with distStrat.scope():
    task = tfm.core.task_factory.get_task(expConfig.task, logging_dir="./train-model/")

In [15]:
for images, labels in task.build_inputs(expConfig.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')


images.shape: (8, 256, 256, 3)  images.dtype: tf.float32
labels.keys: dict_keys(['cls_targets', 'box_targets', 'anchor_boxes', 'cls_weights', 'box_weights', 'image_info'])


In [16]:
categoryIndex = {
	0: {
		'id': 0,
		'name': "speedlimit"
	},
	1: {
		'id': 1,
		'name': "crosswalk"
	},
	2: {
		'id': 2,
		'name': "trafficlight"
	},
	3: {
		'id': 3,
		'name': "stop"
	}
}

tfExDecoder = TfExampleDecoder()

In [17]:
def show_batch(raw_records, num_of_examples):
  plt.figure(figsize=(20, 20))
  use_normalized_coordinates=True
  min_score_thresh = 0.30
  for i, serialized_example in enumerate(raw_records):
    plt.subplot(1, 3, i + 1)
    decoded_tensors = tfExDecoder.decode(serialized_example)
    image = decoded_tensors['image'].numpy().astype('uint8')
    scores = np.ones(shape=(len(decoded_tensors['groundtruth_boxes'])))
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image,
        decoded_tensors['groundtruth_boxes'].numpy(),
        decoded_tensors['groundtruth_classes'].numpy().astype('int'),
        scores,
        category_index=categoryIndex,
        use_normalized_coordinates=use_normalized_coordinates,
        max_boxes_to_draw=200,
        min_score_thresh=min_score_thresh,
        agnostic_mode=False,
        instance_masks=None,
        line_thickness=4)

    plt.imshow(image)
    plt.axis('off')
    plt.title(f'Image-{i+1}')
  plt.show()

In [19]:
buffer_size = 20
num_of_examples = 3

raw_records = tf.data.TFRecordDataset(
    expConfig.task.train_data.input_path).shuffle(
        buffer_size=buffer_size).take(num_of_examples)
show_batch(raw_records, num_of_examples)

/tmp/ipykernel_6975/109230019.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
